<h2 style="margin:.5em 0;">Einführung in Data Science</h2>
<h3 style="margin:.5em 0;">Fakultät Informatik &ndash; Hochschule Aalen</h3>
<h4 style="margin:.5em 0;">Autoren: Florian Merlau; Vithusan Ratneswaran</h4>

#  Bibliotheken importieren 

In [165]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Datenbanken importieren

In [166]:
raw_ehresp = pd.read_csv('ehresp_2016s.dat')

raw_atusrost = pd.read_csv('atusrost_2016.dat')
raw_atussum = pd.read_csv('atussum_2016.dat')
raw_ehresp.head()

,TUCASEID,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,...,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,1,2,26.6,3,3,5,40,0,0,...,1,2,1,2,2,4,2,165,-1,0
1,20160101160066,1,2,44.3,3,3,5,30,15,0,...,1,2,1,-1,1,2,2,300,-1,0
2,20160101160069,1,2,24.5,3,5,-1,75,0,0,...,1,2,1,-1,1,5,1,143,2,0
3,20160101160083,1,2,21.2,1,5,-1,165,0,0,...,1,2,1,2,1,1,2,98,2,0
4,20160101160084,1,1,29.7,3,1,3,30,27,0,...,2,2,3,2,1,1,-1,173,-1,0


# Analyse der Daten

In [167]:
raw_atusrost.columns.values
df_male = raw_atussum[raw_atussum['TESEX'] == 1 ].value_counts().sum() 
df_female = raw_atussum[raw_atussum['TESEX'] == 2].value_counts().sum()


In [168]:
[df_male,df_female]
x=len(raw_atusrost)
y=len(raw_atussum)
z=len(raw_ehresp)
[x,y,z]


[27884, 10493, 10210]

## Fehlerhafte Spalten = 423 --> 423 ungültige Einträge entfernen oder mit geeigneten füllen --> siehe Lösung Aufgabenblatt 1

## Gender-Aufteilung zwischen Mann & Frau  "TESEX =1 --> df_male, "TESEX" = 2 -->df_female 
## Gesamte Länge = 10439
## Länge_Mann = 4670, Länge_Frau, 5823


In [169]:


##df_isna =  raw_atussum[raw_atussum['TESEX'] != 1 & 2].value_counts().sum()
df_isna= raw_atussum.isna().sum()
df_isna= df_isna.value_counts().sum()



In [170]:
df_isnaa= raw_ehresp.isna().sum()
df_isnaa= df_isnaa.value_counts().sum()
df_isnaa


37

## Mergen von Atussum & Ehresp --> aufgrund von fehlendem BMI in atussum 

In [171]:

merged_inner= pd.merge(raw_atussum,raw_ehresp, on = 'TUCASEID', how ='outer')
merged_inner['ERBMI'].isna().sum()
merged_inner.head()




,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,...,1.0,2.0,1.0,2.0,2.0,4.0,2.0,165.0,-1.0,0.0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,...,1.0,2.0,1.0,-1.0,1.0,2.0,2.0,300.0,-1.0,0.0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,...,1.0,2.0,1.0,-1.0,1.0,5.0,1.0,143.0,2.0,0.0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,...,1.0,2.0,1.0,2.0,1.0,1.0,2.0,98.0,2.0,0.0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,...,2.0,2.0,3.0,2.0,1.0,1.0,-1.0,173.0,-1.0,0.0


In [172]:





#

merged_inner_male = merged_inner[merged_inner['TESEX'] == 1 ]
merged_inner_female = merged_inner[merged_inner['TESEX'] == 2]
[merged_inner_male,merged_inner_female]





[             TUCASEID     TUFINLWGT  TRYHHCHILD  TEAGE  TESEX  PEEDUCA  \
 1      20160101160066  5.445941e+06          -1     69      1       37   
 8      20160101160139  9.112257e+06          -1     63      1       46   
 10     20160101160546  3.165768e+06           1     35      1       43   
 16     20160101160573  9.150706e+06          16     16      1       36   
 17     20160101160574  1.299119e+06          13     70      1       40   
 ...               ...           ...         ...    ...    ...      ...   
 10482  20161212162015  2.775790e+06          -1     72      1       40   
 10483  20161212162045  1.985660e+07          17     17      1       37   
 10486  20161212162212  1.986985e+07          -1     75      1       39   
 10489  20161212162357  4.899612e+06          -1     69      1       43   
 10490  20161212162426  1.597792e+07          -1     28      1       43   
 
        PTDTRACE  PEHSPNON  GTMETSTA  TELFS  ...  EUMEAT  EUMILK  EUPRPMEL  \
 1             2    

In [173]:
merged_male=merged_inner_male.drop(columns=['ETWGT', 'EUWGT'] )
merged_female=merged_inner_female.drop(columns=['ETWGT', 'EUWGT'])
merged_inner=
#raw.drop(columns=['SibSp', 'Parch','PassengerId'], inplace=True)
#raw


SyntaxError: invalid syntax (<ipython-input-173-1205a2603d73>, line 3)

## Alle Prädiktionen die mit dem BMI oder Gewicht verbunden sind, wurden gelöscht.

In [71]:
merged_male.columns
merged_female.columns

Index(['TUCASEID', 'TUFINLWGT', 'TRYHHCHILD', 'TEAGE', 'TESEX', 'PEEDUCA',
       'PTDTRACE', 'PEHSPNON', 'GTMETSTA', 'TELFS',
       ...
       'EUINCOME2', 'EUMEAT', 'EUMILK', 'EUPRPMEL', 'EUSODA', 'EUSTORES',
       'EUSTREASON', 'EUTHERM', 'EUWIC', 'EXINCOME1'],
      dtype='object', length=457)

In [147]:
merged_male.describe(include='all').T
merged_male['ERBMI'].isna().sum()
merged_male.isna().sum()


TUCASEID        0
TUFINLWGT       0
TRYHHCHILD      0
TEAGE           0
TESEX           0
             ... 
EUSTORES      127
EUSTREASON    127
EUTHERM       127
EUWIC         127
EXINCOME1     127
Length: 457, dtype: int64

In [148]:
merged_female.describe(include='all').T
merged_female['ERBMI'].isna().sum()
merged_male.isna().sum()

TUCASEID        0
TUFINLWGT       0
TRYHHCHILD      0
TEAGE           0
TESEX           0
             ... 
EUSTORES      127
EUSTREASON    127
EUTHERM       127
EUWIC         127
EXINCOME1     127
Length: 457, dtype: int64

In [75]:
#fig = go.Figure()
#fig.add_trace(go.Histogram(x=df_mean[df_mean['Sex'] == 'male']['Age'], name='Male'))
#fig.add_trace(go.Histogram(x=df_mean[df_mean['Sex'] == 'female']['Age'], name='Female'))
#fig.update_traces(opacity=0.75, xbins=dict(start=10.0,end=70.0,size=5))
#fig.show()

fig= go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=15, end= 65, size=5 ))
fig.show()


In [76]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['TEAGE'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['TEAGE'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=15, end= 65, size=5 ))
fig.show()


In [131]:
def calc_data_stats(df, feature_name):
    return pd.DataFrame(dict(
        feature=feature_name,
        mean=df.mean(),
        median=df.median(),
        #mode=df.mode(numeric_only=True).T[0],
        mode=df.mode(),
        std=df.std(),
    ))
ERBMI_stats = calc_data_stats(merged_inner['ERBMI'],'ERBMI STATS' )
ERBMI_stats

,feature,mean,median,mode,std
0,ERBMI STATS,26.347532,26.5,-1.0,8.59966


In [164]:
df_meanF= merged_female.copy()
print(df_meanF['ERBMI'].mean())
df_meanF['ERBMI'].fillna(df_meanF['ERBMI'].mean(), inplace=True)
df_meanF['ERBMI'].isna().sum()

25.47540144697371


0

In [163]:
df_meanM= merged_male.copy()
print(df_meanM['ERBMI'].mean())
df_meanM['ERBMI'].fillna(df_meanM['ERBMI'].mean(), inplace=True)
df_meanM['ERBMI'].isna().sum()

27.435439137134054


0

In [151]:

df_mean = merged_inner.copy()
df_mean
print(df_mean['ERBMI'].mean())
df_mean['ERBMI'].fillna(df_mean['ERBMI'].mean(), inplace=True)



26.347531831537708


In [144]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=merged_inner[merged_inner['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=15, end= 65, size=5 ))
fig.show()


In [160]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=df_meanM[df_meanM['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=df_meanF[df_meanF['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=15, end= 65, size=5 ))
fig.show()

In [143]:
fig= go.Figure()
fig.add_trace(go.Histogram(x=df_mean[df_mean['TESEX'] == 1] ['ERBMI'] , name='Male'))
fig.add_trace(go.Histogram(x=df_mean[df_mean['TESEX'] == 2] ['ERBMI'] , name='Female'))
fig.update_traces(opacity = 0.75, xbins=dict(start=15, end= 65, size=5 ))
fig.show()


## Zusammenfassung:
## 200 Fehlerhafte bzw fehlende Werte --> Auffüllung dieser Werte durch berechneten Mean-Wert, 